# Introduction to Azure Search Service 👨‍🏫

- Cloud-Based Search Service: Azure Search Service is a cloud search service with built-in AI capabilities.

- AI-Enriched Information: It enriches all types of information to easily identify and explore relevant content at scale.

- Rich Indexing Capabilities: Offers rich indexing with integrated data chunking and vectorization, as well as lexical analysis for text.

- Integration with Azure Services: Can integrate with other Azure services for data ingestion/retrieval and incorporate consumable AI from Azure AI services.

- Generative AI Applications: Optimized for retrieval-augmented generation within Generative AI applications, allowing storage, indexing, and searching of data.

## Import Libraries & Initialize Environment Variables

In [6]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_openai import AzureOpenAIEmbeddings
load_dotenv()

True

## Create Azure OpenAI Embeddings Instance

In [7]:
embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    azure_deployment="embeddings",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

## Create Azure Search Instance

In [8]:
index_name: str = "product-info-test"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=os.getenv("AZURE_SEARCH_ENDPOINT"),
    azure_search_key=os.getenv("AZURE_SEARCH_KEY"),
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

## Load and Chunk Home Theater Product Information

In [9]:
loader = TextLoader("..\sample-docs\hometheatersystem.txt", encoding="utf-8")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=30)
docs = text_splitter.split_documents(documents)
print(docs)

Created a chunk of size 345, which is longer than the specified 200
Created a chunk of size 480, which is longer than the specified 200
Created a chunk of size 1188, which is longer than the specified 200
Created a chunk of size 649, which is longer than the specified 200
Created a chunk of size 1337, which is longer than the specified 200
Created a chunk of size 366, which is longer than the specified 200
Created a chunk of size 395, which is longer than the specified 200
Created a chunk of size 321, which is longer than the specified 200
Created a chunk of size 308, which is longer than the specified 200
Created a chunk of size 222, which is longer than the specified 200
Created a chunk of size 279, which is longer than the specified 200


[Document(page_content='Product Name: Ultimate Home Theater System\n\nBrand: UltraView\n\nModel: UHTS-1000\n\nPrice: $4999', metadata={'source': '..\\sample-docs\\hometheatersystem.txt'}), Document(page_content='Description:\nThe Ultimate Home Theater System (UHTS-1000) is designed for those who demand the highest quality audio and video experience. This state-of-the-art system combines a 75-inch 8K Ultra HD Smart TV with a premium 7.1.4 Dolby Atmos surround sound system to create an immersive home entertainment experience that rivals the best cinemas.', metadata={'source': '..\\sample-docs\\hometheatersystem.txt'}), Document(page_content='TV Specifications:', metadata={'source': '..\\sample-docs\\hometheatersystem.txt'}), Document(page_content='Screen Size: 75 inches\nResolution: 8K Ultra HD (7680 x 4320)\nDisplay Technology: OLED\nHDR: HDR10+, Dolby Vision\nRefresh Rate: 120Hz\nSmart TV: Yes, powered by UltraView OS\nVoice Control: Yes, compatible with Alexa, Google Assistant, and Si

## Add Chunks to Azure Search Index

In [10]:
vector_store.add_documents(documents=docs)

['NGExMzJmYzktNGVjNC00YWJhLWFmNjYtYWVmNDJiODQwYTE3',
 'OTgzODc5MzYtNjQxOS00MGZhLTk1NTItOTI2YmFmNzlkMzM0',
 'ZTZjOWIxNGUtMDA0Ni00MDc0LWFlNTctM2RkMmUxODdiOTA2',
 'ZDYxNzY4ZDMtYmRhMS00OWFjLTk0MGEtNThkYzM2ZTBiMDcw',
 'Y2EwZTFhMWEtMTMwZi00ZDM5LWI3YTUtNjkzMmY0M2MxNmVm',
 'YjMzMjVlOGMtNzNiYS00NDllLWJlYzAtOWM1OGNiNTY2MzBl',
 'NmIzMWU4ZWMtZTYxYS00ZDMwLWI4ZTQtMDgxNTljNGIzZmFh',
 'MWZjMGNmNjQtNmYwMi00NjBhLThlZmEtNzMwMDdjNDFiNjhh',
 'NTJiZGExMWYtN2Q2OC00YTBlLTk3MjItYjdlNTcwZDkwMmI2',
 'MDdjNmExYmQtYTRiYS00MzMyLWExMzUtMTZjNjNmNTYxYjFj',
 'NTk0YTI5ZTEtMmYxMi00OWRiLThlZmYtZGI2YjI4ZDg0ZDFi',
 'YTJkOGE4NTctOTAzYS00MTg1LWI2NWQtMGEyZDU3MmZmMjVh',
 'NTNkOTA3ODAtYzhiYy00OTUzLWExNmYtNGU4NmQ1Yjk1YmY0',
 'NjBkMzhlNzUtN2U3MC00YjZiLThlOTYtOTM0OWNhYmUzODM4',
 'MDk5N2YwNjAtYTRhZC00NDI5LThjMmEtMGQ3ZjQ2ODNhOWMz',
 'OTY1M2FkYjktNTdmOC00MTY4LTg0Y2YtNDlhNzc5N2RkZTkz',
 'NzI3OWMyNmEtZWUwMC00ZWRlLWFjZTktNzFjMWM2ZjhmYTdk',
 'MTU1YzEwYzktYWUwZC00MTVhLTk3MWMtNmJmZDVlYTljYTM3',
 'MGU5MWYzMWQtZWE0Ni00ZDQ1LTg1YzgtNTYyMTNhM2Vj

## Perform a Similarity Search

In [11]:
docs = vector_store.similarity_search(
    query="How much does the home theater system cost?",
    k=3,
    search_type="similarity",
)
print(docs[0].page_content)

Product Name: Ultimate Home Theater System

Brand: UltraView

Model: UHTS-1000

Price: $4999
